<a href="https://colab.research.google.com/github/254francis/Intro_to_AI_Labs/blob/main/Final_final_cotton_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq
from collections import deque

# ─── ENVIRONMENT CLASS ───────────────────────────────────────────────────────
class Environment:
    def __init__(self):
        self.graph = {}
        self.heuristics = {}

    def add_edge(self, from_point, to_point):
        cost = abs(from_point - to_point)  # Energy cost as per assignment
        self.graph.setdefault(from_point, []).append((to_point, cost))
        self.graph.setdefault(to_point, []).append((from_point, cost))  # Bidirectional

    def set_heuristics_zero(self):
        for node in self.graph:
            self.heuristics[node] = 0

# ─── PATH FINDING CLASS ──────────────────────────────────────────────────────
class PathFinder:
    def __init__(self, environment):
        self.env = environment

    # Task i: Fewest stopovers (minimum hops - BFS)
    def fewest_stopovers(self, start, goal):
        visited = set([start])
        queue = deque([(start, [start])])
        while queue:
            node, path = queue.popleft()
            if node == goal:
                return path
            for neighbor, _ in self.env.graph.get(node, []):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, path + [neighbor]))
        return None

    # Task ii: All possible paths (DFS)
    def all_paths(self, start, goal):
        def dfs(node, path, visited, results):
            visited.add(node)
            path.append(node)
            if node == goal:
                results.append(path[:])
            else:
                for neighbor, _ in self.env.graph.get(node, []):
                    if neighbor not in visited:
                        dfs(neighbor, path, visited, results)
            path.pop()
            visited.remove(node)

        results = []
        dfs(start, [], set(), results)
        return results

    # Task iii: Cheapest path (Uniform Cost Search)
    def uniform_cost_search(self, start, goal):
        pq = [(0, start, [])]
        visited = set()
        while pq:
            cost, node, path = heapq.heappop(pq)
            if node == goal:
                return cost, path + [node]
            if node not in visited:
                visited.add(node)
                for neighbor, w in self.env.graph[node]:
                    if neighbor not in visited:
                        heapq.heappush(pq, (cost + w, neighbor, path + [node]))
        return float('inf'), []

    # Task iv: A* Search with heuristic (here, heuristics = 0)
    def a_star(self, start, goal):
        pq = [(self.env.heuristics.get(start, 0), 0, start, [])]
        visited = set()
        while pq:
            est_total, cost_so_far, node, path = heapq.heappop(pq)
            if node == goal:
                return cost_so_far, path + [node]
            if node not in visited:
                visited.add(node)
                for neighbor, w in self.env.graph.get(node, []):
                    if neighbor not in visited:
                        g = cost_so_far + w
                        h = self.env.heuristics.get(neighbor, 0)
                        f = g + h
                        heapq.heappush(pq, (f, g, neighbor, path + [node]))
        return float('inf'), []

# ─── AGENT CLASS ──────────────────────────────────────
class Agent:
    def __init__(self, pathfinder):
        self.pathfinder = pathfinder

    def act(self, mode, start, goal):
        if mode == 'quick-response':
            return self.pathfinder.fewest_stopovers(start, goal)
        elif mode == 'explore':
            return self.pathfinder.all_paths(start, goal)
        elif mode == 'low-battery':
            return self.pathfinder.uniform_cost_search(start, goal)
        elif mode == 'heuristic-optimized':
            return self.pathfinder.a_star(start, goal)
        else:
            raise ValueError("Invalid mode.")

# ─── MAIN DRIVER ────────────
def main():
    env = Environment()
    edges = [
        (0, 1), (0, 6), (0, 7),
        (1, 2), (1, 8),
        (2, 3), (2, 4),
        (3, 9),
        (4, 5),
        (5, 10),
        (6, 11)
    ]
    for u, v in edges:
        env.add_edge(u, v)

    env.set_heuristics_zero()

    pathfinder = PathFinder(env)
    agent = Agent(pathfinder)

    print("Scan Point Navigation System")
    print("Modes: quick-response | explore | low-battery | heuristic-optimized")
    mode = input("Enter agent mode: ").strip()
    start = int(input("Enter start scan point (0-11): ").strip())
    goal = int(input("Enter goal scan point (0-11): ").strip())

    result = agent.act(mode, start, goal)

    if isinstance(result, tuple):
        cost, path = result
        print(f"\nPath: {path} | Total Cost: {cost}")
    elif isinstance(result, list):
        if all(isinstance(p, list) for p in result):  # all paths
            print(f"\nAll possible paths from {start} to {goal}:")
            for p in result:
                print("  →", p)
        else:
            print(f"\nFewest stopovers path: {result}")
    else:
        print("No path found.")

# ─── RUN THE PROGRAM
if __name__ == "__main__":
    main()

Scan Point Navigation System
Modes: quick-response | explore | low-battery | heuristic-optimized
